# Fill in the names

In [30]:
import pandas as pd
import numpy as np
import json
templates = pd.read_json('../templates/name_number_templates.jsonl', lines=True)
fillable_vals = json.load(open('name_tags.json'))

person names

In [31]:
for i in range(len(templates['question'])):
    # there are at most 5 people
    names = np.random.choice(fillable_vals['persons'], 5, replace=False)
    q = templates['question'][i]
    for j in range(1, 6):
        tag  = '{person' + str(j) + '}'
        q = q.replace(tag, names[j-1])
    templates['question'][i] = q

<ipython-input-31-9504520b565f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-31-9504520b565f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-31-9504520b565f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-31-9504520b565f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

places

In [32]:
for i in range(len(templates['question'])):
    # there are at most 3 places
    names = np.random.choice(fillable_vals['places'], 3, replace=False)
    q = templates['question'][i]
    for j in range(1, 4):
        tag  = '{place' + str(j) + '}'
        q = q.replace(tag, names[j-1])
    templates['question'][i] = q

<ipython-input-32-4532f283e149>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-32-4532f283e149>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-32-4532f283e149>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-32-4532f283e149>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

Update foods

In [33]:
for i in range(len(templates['question'])):
    # there are at most 4 foods
    names = np.random.choice(fillable_vals['foods'], 4, replace=False)
    q = templates['question'][i]
    for j in range(1, 5):
        tag  = '{food' + str(j) + '}'
        q = q.replace(tag, names[j-1])
    templates['question'][i] = q

<ipython-input-33-74c049a9ed0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-33-74c049a9ed0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-33-74c049a9ed0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-33-74c049a9ed0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

Update Currencies

In [34]:
for i in range(len(templates['question'])):
    # there are at most 4 foods
    names = np.random.choice(fillable_vals['currencies'], 1, replace=False)
    q = templates['question'][i]
    for j in range(1, 2):
        tag  = '{currencysymbol' + str(j) + '}'
        q = q.replace(tag, names[j-1])
    templates['question'][i] = q

<ipython-input-34-397587e4594e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-34-397587e4594e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-34-397587e4594e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates['question'][i] = q
<ipython-input-34-397587e4594e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [35]:
templates['question'][0]

"Elijah’s ducks lay {x} eggs per day. Elijah eats {y} for breakfast every morning and bakes muffins for her friends every day with {z}. Elijah sells the remainder at the farmers' market daily for €{d} per fresh duck egg. How much in dollars does Elijah make every day at the farmers' market?"

# Fill in the numbers

In [36]:
# note there are only 20 annotated number samples
number_templates = templates.head(20)

In [37]:
import os
from openai import OpenAI
import pandas as pd
import json

# set prompt
with open('prompt_files/fill_in_numbers_prompt.txt', 'r') as file:
    system_prompt = file.read()

suffix = "\n\nYour output should strictly be a json object mapping the inputs to outputs. It should be in the format of {variable1: value, variable2: value, ...}"

# get API key
with open('prompt_files/openai_key.txt', 'r') as file:
    API_KEY = file.read().strip()

def get_variables(target_question, API_KEY = API_KEY, prompt = system_prompt, suffix = suffix):
  os.environ["OPENAI_API_KEY"] = API_KEY
  client = OpenAI()
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": prompt},
          {"role": "user", "content": target_question + suffix},
      ]
  )
  answer = response.choices[0].message.content
  return answer

In [38]:
def valid_output(constraints, vars):
    keys = vars.keys()
    # go in this order to avoid partial matching of future keys
    sorted_keys = sorted(keys, key=lambda x: len(x), reverse=True)
    for constraint in constraints:
        original_constraint = constraint
        for key in sorted_keys:
            constraint = constraint.replace(key, str(vars[key]))
        if not eval(constraint):
            print("Failed constraint: ", original_constraint, "with", constraint)
            return False
    return True

In [39]:
def replace_vars(query, answer, vars):
    for key in vars:
        query = query.replace('{' + key + '}', str(vars[key]))
        answer = answer.replace('{' + key + '}', str(vars[key]))
        # answer = answer.replace(key, str(vars[key]))
    return query, answer

In [40]:
completed_cache = set() # useful for debugging

In [41]:
# max_retries = 10
# for i in range(len(number_templates)):
#     if i in completed_cache:
#         continue
#     print(i)
#     query = str(number_templates.iloc[i].to_dict())
#     num_retries = 0
#     while num_retries < max_retries:
#         vars = get_variables(query)
#         try:
#             vars = json.loads(vars.strip('```').replace('json', '').strip())
#         except:
#             print("invalid json. retrying")
#             num_retries += 1
#             continue
#         if valid_output(number_templates.iloc[i]['constraints'], vars):
#             q, ans = replace_vars(number_templates.iloc[i]['question'], number_templates.iloc[i]['answer'], vars)
#             number_templates.iloc[i]['question'] = q
#             number_templates.iloc[i]['answer'] = ans
#             completed_cache.add(i)
#             break
#         else:
#             print("didn't meet constraints. retrying")
#             num_retries += 1
#     if num_retries == max_retries:
#         print("Failed to generate valid output for template " + str(i))

In [42]:
# number_templates.to_json('../datasets/gsm_name_numbers.jsonl', orient='records', lines=True)

# Do 5 variations per question

In [43]:
import numpy as np
big_df = pd.DataFrame(np.repeat(number_templates.values, 5, axis=0))
big_df.columns = number_templates.columns

In [44]:
completed_cache = set() # useful for debugging

In [45]:
# Do 5 variations per question
import numpy as np
big_df = pd.DataFrame(np.repeat(number_templates.values, 5, axis=0))
big_df.columns = number_templates.columns
max_retries = 10
for i in range(len(big_df)):
    if i in completed_cache:
        continue
    print(i)
    query = str(big_df.iloc[i].to_dict())
    num_retries = 0
    while num_retries < max_retries:
        vars = get_variables(query)
        try:
            vars = json.loads(vars.strip('```').replace('json', '').strip())
        except:
            print("invalid json. retrying")
            num_retries += 1
            continue
        if valid_output(big_df.iloc[i]['constraints'], vars):
            q, ans = replace_vars(big_df.iloc[i]['question'], big_df.iloc[i]['answer'], vars)
            big_df.iloc[i]['question'] = q
            big_df.iloc[i]['answer'] = ans
            completed_cache.add(i)
            break
        else:
            print("didn't meet constraints. retrying")
            num_retries += 1
    if num_retries == max_retries:
        print("Failed to generate valid output for template " + str(i))
big_df.to_json('../data/gsm_name_numbers_100.jsonl', orient='records', lines=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Failed constraint:  h % 1 == 0 with 17.5 % 1 == 0
didn't meet constraints. retrying
Failed constraint:  h % 1 == 0 with 12.5 % 1 == 0
didn't meet constraints. retrying
Failed constraint:  h % 1 == 0 with 17.5 % 1 == 0
didn't meet constraints. retrying
31
32
33
Failed constraint:  h % 1 == 0 with 17.5 % 1 == 0
didn't meet constraints. retrying
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
Failed constraint:  a % f ==  0 with 100 % 30 ==  0
didn't meet constraints. retrying
Failed constraint:  a % f ==  0 with 100 % 30 ==  0
didn't meet constraints. retrying
58
Failed constraint:  a % f ==  0 with 100 % 30 ==  0
didn't meet constraints. retrying
59
60
61
62
63
64
65
66
67
68
69
70
71
72
Failed constraint:  f == a * d with 1500 == 15 * 10
didn't meet constraints. retrying
Failed constraint:  f == a * d with 6000 == 20 * 30
didn't meet constraints. retrying
73
74
75
76
Failed constr